# Food Delivery Data Analysis Notebook

## Load all datasets

In [ ]:

import pandas as pd
import sqlite3

orders = pd.read_csv("orders.csv")
orders["order_date"] = pd.to_datetime(orders["order_date"], dayfirst=True)

users = pd.read_json("users.json")

conn = sqlite3.connect(":memory:")
with open("restaurants.sql", "r") as f:
    conn.executescript(f.read())
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

df = orders.merge(users, on="user_id", how="left")
df = df.merge(restaurants, on="restaurant_id", how="left")

df.head()


## Multiple-choice analysis outputs

In [ ]:

# City with highest Gold revenue
df[df["membership"]=="Gold"].groupby("city")["total_amount"].sum().idxmax()


In [ ]:

# Cuisine with highest average order value
df.groupby("cuisine")["total_amount"].mean().idxmax()


In [ ]:

# Users with total spend > 1000
(df.groupby("user_id")["total_amount"].sum() > 1000).sum()


In [ ]:

# Rating range with highest revenue
bins=[3,3.5,4,4.5,5]
labels=["3.0–3.5","3.6–4.0","4.1–4.5","4.6–5.0"]
df["rating_range"]=pd.cut(df["rating"],bins=bins,labels=labels)
df.groupby("rating_range")["total_amount"].sum().idxmax()


In [ ]:

# City with highest Gold average order value
df[df["membership"]=="Gold"].groupby("city")["total_amount"].mean().idxmax()


In [ ]:

# Cuisine with lowest restaurant count
df.groupby("cuisine")["restaurant_id"].nunique().idxmin()


In [ ]:

# Percentage of Gold orders
round((df[df["membership"]=="Gold"].shape[0]/df.shape[0])*100)


In [ ]:

# Restaurant with highest AOV and <20 orders
stats=df.groupby("restaurant_name").agg(avg=("total_amount","mean"),cnt=("order_id","count"))
stats[stats["cnt"]<20]["avg"].idxmax()


In [ ]:

# Revenue-dominant combination
combo=df.groupby(["membership","cuisine"])["total_amount"].sum().idxmax()
f"{combo[0]} + {combo[1]} cuisine"


In [ ]:

# Highest revenue quarter
df["quarter"]=df["order_date"].dt.to_period("Q")
str(df.groupby("quarter")["total_amount"].sum().idxmax())


## Numerical answers

In [ ]:

df[df["membership"]=="Gold"].shape[0]


In [ ]:

round(df[df["city"]=="Hyderabad"]["total_amount"].sum())


In [ ]:

df["user_id"].nunique()


In [ ]:

round(df[df["membership"]=="Gold"]["total_amount"].mean(),2)


In [ ]:

df[df["rating"]>=4.5].shape[0]


In [ ]:

top_city=df[df["membership"]=="Gold"].groupby("city")["total_amount"].sum().idxmax()
df[(df["membership"]=="Gold") & (df["city"]==top_city)].shape[0]
